In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
    
from matplotlib import pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn import linear_model


In [2]:
#DATA

df= pd.read_csv("D:/Machine Learning Practical/Lab Exercises/Lab 1/Program1/college.csv")
df.head()

,Names,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [3]:
#Setting index as Names of the Universities
college=df.set_index('Names')
college

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Names,,,,,,,,,,,,,,,,,,
Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Worcester State College,No,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
Xavier University,Yes,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
Xavier University of Louisiana,Yes,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49


In [4]:
#changing yes to 1 and no to 0
college = college.replace({'Private': {'Yes': 1, 
                                'No': 0}})
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Names,,,,,,,,,,,,,,,,,,
Abilene Christian University,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,1,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,1,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,1,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


### 1. Fit a ridge regression model on the training set, with λ chosen by cross-validation. Report the test error obtained. 

In [5]:
#Defining X AND Y
X=pd.DataFrame(college.drop("Apps",axis=1)) #Length=777
y=pd.DataFrame(college['Apps']) #Length=777

#Making 10 folds for X and Y
X_folds = np.array_split(X, 10) #lENGTH of 1 fold= 77/78
y_folds = np.array_split(y, 10) #lENGTH of 1 fold= 77/78

RMSE_ridge=[]
MSE_LAMBDA=[]

alphas=np.arange(.001, 10, .01)

for j in alphas:
    for i in range(len(X_folds)):

        # Create training and test sets
        X_test  = X_folds[i] #takes the i'th X fold i.e 1 fold 
        y_test  = y_folds[i] #takes the i'th y fold
        X_train = X.drop(X_folds[i].index) #take everything except the i'th fold i.e 9 folds
        y_train = y.drop(y_folds[i].index) #take everything except the i'th fold i.e 9 folds

        #model
        model = linear_model.Ridge(alpha=j, fit_intercept=True, solver='cholesky').fit(X_train, y_train)

        #pedicting for test
        y_pred_ridge = model.predict(X_test)

        #RMSE
        rmse=np.mean(np.square(np.array(y_pred_ridge)-np.array(y_test)))
        RMSE_ridge.append(rmse)

    MSE=np.mean(RMSE_ridge)
    MSE_LAMBDA.append(MSE)
#the MSE for different values of alpha, we choose the one which gives the least MSE
MSE_LAMBDA

#Least MSE
m=np.min(MSE_LAMBDA)
a=MSE_LAMBDA.index(m)

print("Therefore for the alpha value of",alphas[a],"we get the least MSE i.e",m)


Therefore for the alpha value of 9.990999999999998 we get the least MSE i.e 1287757.914515979


In [6]:
#Fitting the best model,i.e with the alpha value of 10

RMSE_ridge=[]

for i in range(len(X_folds)):

        # Create training and test sets
        X_test  = X_folds[i] #takes the i'th X fold i.e 1 fold 
        y_test  = y_folds[i] #takes the i'th y fold
        X_train = X.drop(X_folds[i].index) #take everything except the i'th fold i.e 9 folds
        y_train = y.drop(y_folds[i].index) #take everything except the i'th fold i.e 9 folds

        #model
        model = linear_model.Ridge(alpha=0.9909999999999999, fit_intercept=True, solver='cholesky').fit(X_train, y_train)

        #pedicting for test
        y_pred_ridge = model.predict(X_test)

        #RMSE for test
        rmse=np.mean(np.square(np.array(y_pred_ridge)-np.array(y_test)))
        RMSE_ridge.append(rmse)

                
#test error ridge
test_error_ridge=np.mean(RMSE_ridge)
test_error_ridge

1288157.2854329185

### 2. Fit a lasso model on the training set, with λ chosen by cross validation. Report the test error obtained, along with the number of non-zero coefficient estimates.

In [7]:
#Defining X AND Y
X=pd.DataFrame(college.drop("Apps",axis=1)) #Length=777
y=pd.DataFrame(college['Apps']) #Length=777

#Making 10 folds for X and Y
X_folds = np.array_split(X, 10) #lENGTH of 1 fold= 77/78
y_folds = np.array_split(y, 10) #lENGTH of 1 fold= 77/78

RMSE_lasso=[]
MSE_LAMBDA_lasso=[]

alphas=np.arange(.001, 1, .01)

for j in alphas:
    for i in range(len(X_folds)):

        # Create training and test sets
        X_test  = X_folds[i] #takes the i'th X fold i.e 1 fold 
        y_test  = y_folds[i] #takes the i'th y fold
        X_train = X.drop(X_folds[i].index) #take everything except the i'th fold i.e 9 folds
        y_train = y.drop(y_folds[i].index) #take everything except the i'th fold i.e 9 folds

        #model
        model = linear_model.Lasso(alpha=j, fit_intercept=True, selection='cyclic').fit(X_train, y_train)

        #pedicting for test
        y_pred_lasso = model.predict(X_test)

        #RMSE
        rmse=np.mean(np.square(np.array(y_pred_lasso)-np.array(y_test)))
        RMSE_lasso.append(rmse)

    MSE=np.mean(RMSE_lasso)
    MSE_LAMBDA_lasso.append(MSE)
#the MSE for different values of alpha, we choose the one which gives the least MSE
MSE_LAMBDA

#Least MSE
m=np.min(MSE_LAMBDA_lasso)
a=MSE_LAMBDA_lasso.index(m)

print("Therefore for the alpha value of",alphas[a],"we get the least MSE i.e",m)

Therefore for the alpha value of 0.001 we get the least MSE i.e 26265400.259337436


In [8]:
#The best model can be fitted for alpha=0.001

RMSE_lasso=[]
MSE_LAMBDA_lasso=[]

for i in range(len(X_folds)):

        # Create training and test sets
        X_test  = X_folds[i] #takes the i'th X fold i.e 1 fold 
        y_test  = y_folds[i] #takes the i'th y fold
        X_train = X.drop(X_folds[i].index) #take everything except the i'th fold i.e 9 folds
        y_train = y.drop(y_folds[i].index) #take everything except the i'th fold i.e 9 folds

        #model
        model = linear_model.Lasso(alpha=0.001, fit_intercept=True, selection='cyclic').fit(X_train, y_train)

        #pedicting for test
        y_pred_lasso = model.predict(X_test)

        #RMSE
        rmse=np.mean(np.square(np.array(y_pred_lasso)-np.array(y_test)))
        RMSE_lasso.append(rmse)
        
#test error lasso
test_error_lasso=np.mean(RMSE_lasso)
test_error_lasso

26265400.259337436

In [9]:
#Getting the Model Coefficients
model.coef_

array([-4.85970349e+02,  1.60180299e+00, -9.28211966e-01,  4.88547954e+01,
       -1.33212249e+01,  6.16173572e-02,  4.07630646e-02, -9.41983172e-02,
        1.37665906e-01,  4.49105588e-02,  1.27486605e-02, -1.01463512e+01,
       -2.56269723e+00,  1.93879952e+01,  4.30181513e-01,  9.42918420e-02,
        9.07949903e+00])